In [1]:
import nltk
from nltk import word_tokenize
from matplotlib import pyplot as plt
from wordcloud import WordCloud
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shreeyachand/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd
import datetime

In [9]:
urls = [
    'https://www.bbb.org/us/tn/memphis/profile/apartments/the-residences-at-lakeview-0543-44158002',
    'https://www.bbb.org/us/tn/memphis/profile/apartments/mill-creek-apartments-0543-44074598',
    'https://www.bbb.org/us/tn/germantown/profile/property-management/maa-0543-31000112',
    'https://www.bbb.org/us/tn/memphis/profile/apartments/the-trails-at-mt-moriah-0543-44120390',
    'https://www.bbb.org/us/tn/memphis/profile/apartments/winchester-grove-apartments-0543-44141723'
]

In [8]:
def get_df(u):
    return u.split('-0543')[0].split('/')[-1]

In [23]:
xls = pd.ExcelFile('bbb-complaints-clean.xlsx')
dfs = []
for x in map(get_df, urls):
    dfs.append((x, pd.read_excel(xls, x)))
    
dfs.append(('other', pd.read_excel(xls, 'other')))

In [24]:
nltk.download("stopwords")
from nltk.corpus import stopwords
stopwords_list = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreeyachand/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# create a large list of stopwords from multiple sources to omit from word clouds
stop = []
with open('stopwords.txt') as f:
    lines = f.readlines()
    for line in lines:
        word = line.strip('\n')
        if word not in stopwords_list and word not in stop:
            stop.append(word)
    
with open('stopwords-en.txt') as f:
    lines = f.readlines()
    for line in lines:
        word = line.strip('\n')
        if word not in stopwords_list and word not in stop:
            stop.append(word)
stopwords_list.extend(stop)

In [27]:
for owner in dfs:
    caps = ""
    py = owner[1]
    for row in py['text']:
        caps += row + ' '
        caps = caps.encode('utf-8').decode('utf-8')
        words = word_tokenize(caps)
        words_no_punc = []
    for word in words:
        if word.isalpha():
            words_no_punc.append(word.lower())
    clean_words = []
    for word in words_no_punc:
        if word not in stopwords_list:
            clean_words.append(word)
            clean_words_string = " ".join(clean_words)
    wordcloud = WordCloud(width=800, height=400).generate(clean_words_string)
    wordcloud.to_file(owner[0]+'-bbb-wordcloud.png')

In [28]:
google = pd.read_excel('google_review_data_full.xlsx',index_col=0)

In [49]:
landlords = list(set(google['name_of_landlord']))
landlords

['THE TRAILS AT MT MORIAH',
 'THE VIEW AT SHELBY FARM',
 'KENSINGTON MANOR APARTMENTS',
 'LINCOLN ON THE GREEN',
 'WINCHESTER GROVE APARTMENTS',
 'UNIVERSITY GARDENS MANOR',
 'THE PARK AT NEW CASTLE',
 'MILLCREEK APTS., LP',
 'The Residences at Lakeview']

In [58]:
for ll in landlords:
    caps = ""
    py = google[google['name_of_landlord'] == ll]
    py = py[py['text_of_review'] != 'No text review found']
    ll = '-'.join(i for i in list(map(lambda x: x.strip('., lp'), ll.lower().split(' '))) if i != '')
    for row in py['text_of_review']:
        caps += str(row) + ' '
        caps = caps.encode('utf-8').decode('utf-8')
        words = word_tokenize(caps)
        words_no_punc = []
    for word in words:
        if word.isalpha():
            words_no_punc.append(word.lower())
    clean_words = []
    for word in words_no_punc:
        if word not in stopwords_list:
            clean_words.append(word)
            clean_words_string = " ".join(clean_words)
    wordcloud = WordCloud(width=800, height=400).generate(clean_words_string)
    wordcloud.to_file(ll+'-google-wordcloud.png')